In [1]:
# %pip install -q llama-index
# %pip install -q python-dotenv
# %pip install -q llama-index-embeddings-gemini
# %pip install -q llama-index-llms-gemini
# %pip install -q ipywidgets

In [2]:
from llama_index.core.agent import ReActAgent
from llama_index.llms.gemini import Gemini
from llama_index.core.tools import FunctionTool

In [3]:
from dotenv import load_dotenv
import os

load_dotenv()

GOOGLE_API_KEY = os.environ.get('GOOGLE_API_KEY')
LLAMA_CLOUD_API_KEY = os.environ.get('LLAMA_CLOUD_API_KEY')

In [11]:
from llama_index.embeddings.gemini import GeminiEmbedding
from llama_index.core import Settings

model_name = "models/embedding-001"

Settings.llm = llm =  Gemini(model='models/gemini-1.5-flash',temperature=0,api_key=GOOGLE_API_KEY)

Settings.embed_model = em = GeminiEmbedding(
    model_name=model_name,
    api_key=GOOGLE_API_KEY
)


In [5]:
from llama_index.core import SimpleDirectoryReader, VectorStoreIndex, Settings

documents = SimpleDirectoryReader("./data").load_data()
index = VectorStoreIndex.from_documents(documents)
query_engine = index.as_query_engine()

response = query_engine.query(
    "How much exactly was allocated to a tax credit to promote investment in green technologies in the 2023 Canadian federal budget?"
)
print(response)

This information is not provided in the context you have given. 



In [6]:
from llama_parse import LlamaParse

import nest_asyncio
nest_asyncio.apply()

documents2 = LlamaParse(result_type="markdown").load_data(
    "./data/2023_canadian_budget.pdf"
)
index2 = VectorStoreIndex.from_documents(documents2)
query_engine2 = index2.as_query_engine()

response2 = query_engine2.query(
    "How much exactly was allocated to a tax credit to promote investment in green technologies in the 2023 Canadian federal budget?"
)
print(response2)

Started parsing the file under job_id b2c30b61-481e-4d00-b0f8-d5a48e5a22a6
$20 billion. 



In [7]:
def multiply(a: float, b: float) -> float:
    """Multiply two numbers and returns the product"""
    return a * b


multiply_tool = FunctionTool.from_defaults(fn=multiply)


def add(a: float, b: float) -> float:
    """Add two numbers and returns the sum"""
    return a + b


add_tool = FunctionTool.from_defaults(fn=add)

In [8]:
from llama_index.core.tools import QueryEngineTool

budget_tool = QueryEngineTool.from_defaults(
    query_engine, 
    name="canadian_budget_2023",
    description="A RAG engine with some basic facts about the 2023 Canadian federal budget. Ask natural-language questions about the budget."
)

In [12]:
agent = ReActAgent.from_tools([multiply_tool,add_tool,budget_tool],llm=llm,verbose=True)

In [13]:
import time

time.sleep(5) 

response = agent.chat(
    "How much exactly was allocated to a tax credit to promote investment in green technologies in the 2023 Canadian federal budget?"
)

print(response)

time.sleep(5) 

response = agent.chat(
    "How much was allocated to a implement a means-tested dental care program in the 2023 Canadian federal budget?"
)

print(response)

time.sleep(5) 

response = agent.chat(
    "How much was the total of those two allocations added together? Use a tool to answer any questions."
)

print(response)

> Running step 03f6360e-c492-4079-85a7-c5bc11d7bfea. Step input: How much exactly was allocated to a tax credit to promote investment in green technologies in the 2023 Canadian federal budget?
Thought: The current language of the user is: English. I need to use a tool to help me answer the question.
Action: canadian_budget_2023
Action Input: {'input': 'How much was allocated to a tax credit to promote investment in green technologies in the 2023 Canadian federal budget?'}
Observation: This information is not provided in the context you have given. 

> Running step c50bab05-b3e9-4697-9c1d-dfa18ca4df9b. Step input: None
Thought: I cannot answer the question with the provided tools.
Answer: I'm sorry, but the information about the exact amount allocated to a tax credit for green technologies in the 2023 Canadian federal budget is not available in the provided context.
I'm sorry, but the information about the exact amount allocated to a tax credit for green technologies in the 2023 Canadia